In [2]:
import torch
import torch.nn as nn
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit


ModuleNotFoundError: No module named 'scipy'

In [7]:
G1 = 0.0
F2 = 0.0
F1 = np.linspace(0.001,0.01,5)
G2 = np.linspace(0.001,0.01,5)

array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
       0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
       0.018, 0.019, 0.02 ])

In [ ]:
G1 = 0.0
F2 = 0.0
F1 = np.linspace(0.001,0.01,5)
G2 = np.linspace(0.001,0.01,5)
input_gamma1 = np.linspace(0.0,0.02,21)
measured_gamma1 = np.zeros((len(input_gamma1),len(F1),len(G2)),dtype=np.float64)
gamma1_bias = np.zeros_like(measured_gamma1)
additive_bias = np.zeros((len(F1),len(G2)),dtype=np.float64)
multiplicative_bias = np.zeros((len(F1),len(G2)),dtype=np.float64)
for i in range(len(input_gamma1)):
    measured_gamma1[i][:][:] = fits.getdata(f'data/gamma1_measured_flexion_F1G2_{input_gamma1[i]:.3f}.fits')
    gamma1_bias[i][:][:] = measured_gamma1[i][:][:] - input_gamma1[i]

### use linear fit to get additive bias
for i in range(len(F1)):
    for j in range(len(G2)):
        model = np.poly1d(np.polyfit(input_gamma1, gamma1_bias[:,i,j], 1))
        additive_bias[i,j] = model[0]
        multiplicative_bias[i,j] = model[1]

In [ ]:
f1,g2 = np.meshgrid(F1,G2)
def F1G2(F1G2,a):
    F1,G2 = F1G2
    return a * F1 * G2
F1G2 = np.vstack([f1.ravel(), g2.ravel()])
MB = multiplicative_bias.ravel()

popt, _ = curve_fit(F1G2, F1G2, MB)

# Visualize
a = popt[0]
MB_fit = a * f1 * g2

In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.title("Original Data")
plt.pcolormesh(f1, g2, multiplicative_bias, shading='auto')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title("Fitted Data")
plt.pcolormesh(X, Y, MB_fit, shading='auto')
plt.colorbar()

plt.show()